In [5]:
import requests
import pandas as pd
import numpy as np
import time
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from konlpy.tag import Okt

In [6]:
options = webdriver.ChromeOptions()
#options.add_argument('headless')  # headless를 주면 동작하는 모습을 볼 수 없음
options.add_argument('disable-gpu')
options.add_argument('lang=ko_KR')
driver = webdriver.Chrome('chromedriver', options=options)
driver.implicitly_wait(3)

In [7]:
url='https://www.booking.com/searchresults.ko.html?aid=376440&label=bdot-HhWTaX_r6btPip7sfrk7SQS267777897793%3Apl%3Ata%3Ap1%3Ap22%2C563%2C000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-324456682700%3Alp1009871%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YUlRwjG4dAJkHxCuUKVzpFo&sid=3c50861874335f98b91a4a71fdc2254b&tmpl=searchresults&ac_click_type=b&ac_position=0&class_interval=1&dest_id=-716583&dest_type=city&from_sf=1&group_adults=2&group_children=0&iata=SEL&label_click=undef&no_rooms=1&raw_dest_type=city&room1=A%2CA&sb_price_type=total&search_selected=1&shw_aparth=1&slp_r_match=0&src=searchresults&srpvid=98411b83c6d80043&ss=%EC%84%9C%EC%9A%B8%2C%20%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD&ss_raw=%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD%20%EC%84%9C%EC%9A%B8&ssb=empty&ssne=%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD&ssne_untouched=%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD&top_ufis=1&nflt=class%3D3%3Bclass%3D4%3Bclass%3D5%3Bht_id%3D204%3B&percent_htype_hotel=1&rsf='

In [ ]:
#seoul hotel

hotel_names = []
hotel_tourlists = []
hotel_reviews = []
driver.get(url)

for i in range (1, 27): # 호텔수
    try:
        driver.find_element_by_xpath(
        '//*[@id="hotellist_inner"]/div[{0}]/div[2]/div[1]/div[1]/div[1]/h3/a/span[1]'.format(i)).click() #호텔 클릭        
              
        time.sleep(3.0)
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(3.0)
        hotel_name = driver.find_element_by_xpath('//*[@id="hp_hotel_name"]').text #호텔 이름
        print(hotel_name)
        hotel_tourlist = driver.find_element_by_xpath(
            '//div[@class="hp_location_block__content_container hp-poi-content-container--column"]').text #명소 크롤링
        print(hotel_tourlist)
        try:
            driver.find_element_by_xpath('//*[@id="show_reviews_tab"]/span').click() #리뷰 더 보기 클릭
            time.sleep(1.0)
            driver.find_element_by_xpath(
                '//*[@id="review_list_score_container"]/div[3]/div[1]/div[2]/div/div/label[1]/span').click() #한국어 클릭
        except:
            continue
        for j in range(2, 16): #리뷰 페이지
            try:
                for y in range(1,11):
                    try:
                        hotel_review = driver.find_element_by_xpath(
                            '//*[@id="review_list_page_container"]/ul/li[{0}]/div/div[4]'.format(y)).text #고객후기
                        print(hotel_review)
                        hotel_names.append(hotel_name)
                        hotel_tourlists.append(hotel_tourlist)
                        hotel_reviews.append(hotel_review)
                    except:
                        print('g')
                driver.find_element_by_xpath('//*[@id="review_list_page_container"]/div[4]/div/div[1]/div/div[3]').click() #다음버튼 클릭
                time.sleep(3.0)
            except:
                print('E')
                continue
        driver.close()
        driver.switch_to.window(driver.window_handles[0])

    except:
        print('d')
        continue
driver.close()

In [ ]:
df_review = pd.DataFrame({'hotel_name':hotel_names, 'hotel_review':hotel_reviews, 'hotel_tourlist':hotel_tourlists})
df_review.head()
#df_review.tail()
df_review.info()

In [ ]:
df_review.to_csv('./hotel_review/review_seoul.csv')